# llama3-hackathon

In [ ]:
! pip install llama-index
! pip install pydantic
! pip install groq
! pip install llama-index-llms-groq
! pip install substrate
! pip install yfinance
! pip install pandas
! pip install streamlit
! pip install llama-index-llms-together

In [50]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.llms.groq import Groq
from google.colab import userdata

from llama_index.core.llms import ChatMessage, MessageRole

import yfinance as yf
from typing import Annotated, Callable, Any, Optional
from pandas import DataFrame
from functools import wraps

In [51]:
# Utility function for saving output
def save_output(
    data: DataFrame, description: str, save_path: Optional[str] = None
) -> None:
    if save_path:
        data.to_csv(save_path)
        print(f"{description} saved to {save_path}")


def get_stock_data(
    symbol: str, start_date: str, end_date: str, save_path: Optional[str] = None
) -> str:
    ticker = yf.Ticker(symbol)
    stock_data = ticker.history(start=start_date, end=end_date)
    save_output(stock_data, f"Stock data for {ticker.ticker}", save_path)
    return stock_data.to_string()


def get_stock_info(symbol: str) -> str:
    ticker = yf.Ticker(symbol)
    info = ticker.info
    return str(info)


def get_company_info(symbol: str, save_path: Optional[str] = None) -> str:
    ticker = yf.Ticker(symbol)
    info = ticker.info
    company_info = {
        "Company Name": info.get("shortName", "N/A"),
        "Industry": info.get("industry", "N/A"),
        "Sector": info.get("sector", "N/A"),
        "Country": info.get("country", "N/A"),
        "Website": info.get("website", "N/A"),
    }
    company_info_df = DataFrame([company_info])
    if save_path:
        company_info_df.to_csv(save_path)
        print(f"Company info for {ticker.ticker} saved to {save_path}")
    return company_info_df.to_string()


def get_stock_dividends(symbol: str, save_path: Optional[str] = None) -> str:
    ticker = yf.Ticker(symbol)
    dividends = ticker.dividends
    if save_path:
        dividends.to_csv(save_path)
        print(f"Dividends for {ticker.ticker} saved to {save_path}")
    return dividends.to_string()


def get_income_stmt(symbol: str) -> str:
    ticker = yf.Ticker(symbol)
    income_stmt = ticker.financials
    return income_stmt.to_string()


def get_balance_sheet(symbol: str) -> str:
    ticker = yf.Ticker(symbol)
    balance_sheet = ticker.balance_sheet
    return balance_sheet.to_string()


def get_cash_flow(symbol: str) -> str:
    ticker = yf.Ticker(symbol)
    cash_flow = ticker.cashflow
    return cash_flow.to_string()


def get_analyst_recommendations(symbol: str) -> str:
    ticker = yf.Ticker(symbol)
    recommendations = ticker.recommendations
    if recommendations.empty:
        return "No recommendations available"

    recommendation_counts = recommendations[
        ["strongBuy", "buy", "hold", "sell", "strongSell"]
    ].sum()
    most_common_recommendation = recommendation_counts.idxmax()
    count = recommendation_counts.max()

    return f"Most common recommendation: {most_common_recommendation} ({count} votes)"

In [52]:
# Set up the Groq class with the required model and API key
# llm = Groq(model="llama3-70b-8192", api_key="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")

llm3 = TogetherLLM(
    model="meta-llama/Meta-Llama-3-8B-Instruct-Lite",
    api_key="d0ca8bc44aac868cafe8e100a0e3698183000c8562dc76a286cb96f1b9d8cdc6",
)

In [ ]:
api_key = "apik_tMzRgPeGtRkF9fSWW8vSr0baSgz6wbRU"
if api_key is None:
    raise EnvironmentError("No SUBSTRATE_API_KEY set")

from substrate import Substrate, ComputeText, sb

substrate = Substrate(api_key=api_key, timeout=60 * 5)

story = ComputeText(prompt="tell me a story", model="Llama3Instruct8B")
summary = ComputeText(
    prompt=sb.concat("Summarize this story: ", story.future.text),
    model="Llama3Instruct8B",
)
response = substrate.run(story)

story_out = response.get(story)
print(story_out.text)

In [54]:
# Define tools for each function
get_stock_data_tool = FunctionTool.from_defaults(fn=get_stock_data)
get_stock_info_tool = FunctionTool.from_defaults(fn=get_stock_info)
get_company_info_tool = FunctionTool.from_defaults(fn=get_company_info)
get_stock_dividends_tool = FunctionTool.from_defaults(fn=get_stock_dividends)
get_income_stmt_tool = FunctionTool.from_defaults(fn=get_income_stmt)
get_balance_sheet_tool = FunctionTool.from_defaults(fn=get_balance_sheet)
get_cash_flow_tool = FunctionTool.from_defaults(fn=get_cash_flow)
get_analyst_recommendations_tool = FunctionTool.from_defaults(
    fn=get_analyst_recommendations
)


GPT_FINANCIAL_ANALYST_SYS_STR = """\
You are a Financial Analyst, specializing in providing comprehensive financial analysis and advice. Your role is to assist users with various financial tasks and queries, including but not limited to:

1) **Financial Analysis**:
   - Perform financial statement analysis (balance sheet, income statement, cash flow statement).
   - Conduct ratio analysis (liquidity, profitability, solvency, and efficiency ratios).
   - Evaluate financial performance and trends over time.
   - Analyze financial metrics to assess company health and performance.

2) **Investment Analysis**:
   - Provide insights on stock, bond, and other investment opportunities.
   - Perform risk assessment and return analysis for potential investments.
   - Utilize financial models (DCF, CAPM, etc.) to value assets.
   - Offer portfolio management advice and strategies.

3) **Budgeting and Forecasting**:
   - Assist in creating and managing budgets.
   - Provide financial forecasting based on historical data and market trends.
   - Analyze variances between budgeted and actual figures.

4) **Market Analysis**:
   - Conduct market research to identify economic trends and market conditions.
   - Analyze the impact of macroeconomic factors on financial performance.
   - Provide competitive analysis and industry benchmarking.

5) **Financial Planning and Advisory**:
   - Offer personal financial planning advice, including retirement planning, tax strategies, and estate planning.
   - Advise on debt management and financial risk mitigation.
   - Help with financial goal setting and achievement strategies.

**Guidelines**:
- Stick strictly to the financial analyst role; do not deviate into unrelated areas.
- Use appropriate financial terminology and concepts.
- Provide detailed and accurate analyses and explanations.
- Ensure all advice is based on sound financial principles and current market conditions.
- Prioritize clarity, conciseness, and comprehensiveness in all responses.
"""


prefix_msgs = [
    ChatMessage(role=MessageRole.SYSTEM, content=GPT_FINANCIAL_ANALYST_SYS_STR)
]


aa = ReActAgent.from_tools(
    [
        get_stock_data_tool,
        get_stock_info_tool,
        get_company_info_tool,
        get_stock_dividends_tool,
        get_income_stmt_tool,
        get_balance_sheet_tool,
        get_cash_flow_tool,
        get_analyst_recommendations_tool,
    ],
    llm=llm,
    prefix_messages=prefix_msgs,
    token_counting=True,
    verbose=False,
    max_iterations=3,
)

In [ ]:
# @markdown ---
# @markdown ### Enter a Financial Query:
query = ""  # @param {type:"string"}
response = aa.chat(query)
print(response)

In [3]:
import streamlit as st

# Streamlit UI
st.title("Financial Analyst")

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display conversation history
for message in st.session_state.messages:
    st.write(f"{message['role']}: {message['content']}")

# User input
user_input = st.text_input("You: ", "")

if user_input:
    st.session_state.messages.append({"role": "User", "content": user_input})

    # Get response from the agent
    response = aa.chat(user_input)
    st.session_state.messages.append({"role": "Agent", "content": response})

    # Clear user input
    st.experimental_rerun()

2024-07-20 21:48:34.870 
  command:

    streamlit run /usr/lib/python3/dist-packages/ipykernel_launcher.py [ARGUMENTS]
2024-07-20 21:48:34.876 Session state does not function when running a script without `streamlit run`
